In [1]:
import torch, pandas as pd, numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from skimage import transform
import os
import time
import copy
import torchvision
from torchvision import transforms, utils, models
plt.ion()

In [2]:
class AVAImagesDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.ava_frame = pd.read_csv(csv_file, sep=" ", header=None)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.ava_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, str(self.ava_frame.iloc[idx, 0]) + '.jpg')
        if not os.path.isfile(img_name):
            return None
        image = Image.open(img_name).convert('RGB')
        # classes = np.array([self.ava_frame.iloc[idx, 2:12]])
        classes_txt = self.ava_frame.iloc[idx, 12:14]
        classes = np.zeros((66))
        # print(classes_txt)
        for c in classes_txt:
            # print(c)
            if c != 0:
                classes[(int(c) - 1)] = 1
        classes = classes.astype('float').reshape(-1, 66)
        # print(classes)
        if self.transform:
            image = self.transform(image)
            classes = torch.from_numpy(classes)
            # print(classes)
        sample = {'image': image, 'classes': classes}
        print(sample['image'])
        return sample

In [17]:
# mo_dataset = AVAImagesDataset(csv_file='/media/matt/New Volume/ava/cleanedlabels.txt', 
#                                root_dir='/media/matt/New Volume/missourian_pics/2017/Fall/Dump/Compton, Nate/',
#                                transform=transforms.Compose([
#                                    transforms.Resize(256), 
#                                    transforms.CenterCrop(224), 
#                                    transforms.ToTensor(), 
#                                    transforms.Normalize(
#                                        mean=[0.485, 0.456, 0.406], 
#                                        std=[0.229, 0.224, 0.225])
#                                        ]))

# dataset_size = len(mo_dataset)

# dataloader = DataLoader(mo_dataset, batch_size=1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform=transforms.Compose([transforms.Resize(256), 
                              transforms.CenterCrop(224), 
                              transforms.ToTensor(), 
                              transforms.Normalize(
                                mean=[0.485, 0.456, 0.406], 
                                std=[0.229, 0.224, 0.225])
                              ])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = transform(image).float()
    return image.to(device) #assumes that you're using GPU

model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
# model_ft.fc = nn.Linear(num_ftrs, 10)
model_ft.fc = nn.Linear(num_ftrs, 66)

# model_ft.load_state_dict(torch.load("../common/Nov17_r50_6ep_32batch.pt"))
model_ft.load_state_dict(torch.load("../common/Nov18_r50_20ep_classifier.pt"))
model_ft.eval()
model_ft = model_ft.to(device)

# image = image_loader('/media/matt/New Volume/missourian_pics/2017/Fall/Dump/Compton, Nate/20170824_MUfootballpractice_NC/20170824_mufootball_NC_001.JPG')
image = image_loader('/media/matt/New Volume/missourian_pics/2017/Fall/Dump/Compton, Nate/20170824_MUfootballpractice_NC/20170824_mufootball_NC_008.JPG')
output = model_ft(image[None, ...])
_, preds = torch.max(output, 1)
# print(preds[0].item())
print(preds[0].item() + 1)

1
